In [8]:
!pip install -q mediapipe

In [13]:
!curl -o pose_landmarker_heavy.task https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1 29.2M    1  466k    0     0   564k      0  0:00:53 --:--:--  0:00:53  565k
  6 29.2M    6 1806k    0     0   953k      0  0:00:31  0:00:01  0:00:30  955k
 10 29.2M   10 3134k    0     0  1107k      0  0:00:27  0:00:02  0:00:25 1108k
 13 29.2M   13 4062k    0     0  1061k      0  0:00:28  0:00:03  0:00:25 1062k
 16 29.2M   16 4814k    0     0   997k      0  0:00:30  0:00:04  0:00:26  998k
 18 29.2M   18 5406k    0     0   926k      0  0:00:32  0:00:05  0:00:27  986k
 21 29.2M   21 6488k    0     0   949k      0  0:00:31  0:00:06  0:00:25  948k
 26 29.2M   26 7838k    0     0   999k      0  0:00:29  0:00:07  0:00:22  937k
 29 29.2M   29 8814k    0     0   998k      0  0:00:30  0:00:08  0:00:22  949k
 33 29.2M   33 9966k    0     0  1012k      0  0:00

In [57]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import json
import csv
import os

In [58]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image

In [59]:
def extract_landmarks(detection_result):
    landmarks = []
    for pose_landmarks in detection_result.pose_landmarks:
        for landmark in pose_landmarks:
            landmarks.append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z,
                'visibility': landmark.visibility
            })
    return landmarks

In [60]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Ruta base donde están las carpetas de datos
base_path = 'datos_Mateo'

# Iterar sobre cada categoría de movimiento en la carpeta base
for category in os.listdir(base_path):
    category_path = os.path.join(base_path, category)
    
    # Revisar que sea una carpeta
    if not os.path.isdir(category_path):
        continue
    
    # Crear carpeta de salida para JSON y CSV de la categoría si no existe
    output_json_path = os.path.join(category_path, 'json')
    output_csv_path = os.path.join(category_path, 'csv')
    os.makedirs(output_json_path, exist_ok=True)
    os.makedirs(output_csv_path, exist_ok=True)
    
    # Procesar cada video en la categoría
    for video_file in os.listdir(category_path):
        if not video_file.endswith('.mp4'):
            continue
        
        video_path = os.path.join(category_path, video_file)
        cap = cv2.VideoCapture(video_path)
        all_landmarks = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            detection_result = detector.detect(mp_image)

            landmarks = extract_landmarks(detection_result)
            all_landmarks.append(landmarks)

        cap.release()

        # Guardar los datos en formato JSON
        json_output_path = os.path.join(output_json_path, f"{video_file.split('.')[0]}.json")
        with open(json_output_path, 'w') as json_file:
            json.dump(all_landmarks, json_file, indent=4)

        # Guardar los datos en formato CSV
        csv_output_path = os.path.join(output_csv_path, f"{video_file.split('.')[0]}.csv")
        csv_columns = ['frame', 'landmark_index', 'x', 'y', 'z', 'visibility']
        with open(csv_output_path, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for frame_index, frame_landmarks in enumerate(all_landmarks):
                for landmark_index, landmark in enumerate(frame_landmarks):
                    row = {
                        'frame': frame_index,
                        'landmark_index': landmark_index,
                        'x': landmark['x'],
                        'y': landmark['y'],
                        'z': landmark['z'],
                        'visibility': landmark['visibility']
                    }
                    writer.writerow(row)
